In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities._credentials import UserIdentityConfiguration
from azure.ai.ml.entities._indexes import ModelConfiguration, LocalSource, AzureAISearchConfig, AISearchSource, GitSource
import os

: 

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

client=MLClient(DefaultAzureCredential(), 
                subscription_id="f375b912-331c-4fc5-8e9f-2d7205e3e036",
                resource_group_name="rg-jingyizhuai",
                workspace_name="jingyizhu-2148")


from azure.ai.ml.entities import Index
client.indexes.create_or_update(Index(name="new-mlindex", path="MLIndex", version="1", stage="Development"))
ml_index=client.indexes.get("new-mlindex", label="latest")

retriever = ml_index.as_langchain_retriever()
retriever.get_relevant_documents("which tent is the most waterproof?")

In [ ]:
# client=MLClient(DefaultAzureCredential(), 
#                 subscription_id="f375b912-331c-4fc5-8e9f-2d7205e3e036",
#                 resource_group_name="rg-jingyizhuaiint",
#                 workspace_name="jingyizhu-2543")
# client=MLClient(DefaultAzureCredential(), 
#                 subscription_id="f375b912-331c-4fc5-8e9f-2d7205e3e036",
#                 resource_group_name="rg-jingyizhuaiint",
#                 workspace_name="jingyizhu-2543")

client=MLClient.from_config(DefaultAzureCredential(), path="./config.json")
# index_name = "remote-ai-search-00"
index_name = "remote-cohere-00"

## aoai and acs connections - entra id
aoai_connection = client.connections.get("jingyiaoai")
ai_search_connection = client.connections.get("jingyiacsuk")
embeddings_model_config = ModelConfiguration.from_connection(aoai_connection, 
                                                             model_name="text-embedding-ada-002",
                                                             deployment_name="text-embedding-ada-002")

## aoai and acs connections
aoai_connection = client.connections.get("jingyizhuoai", populate_secrets=True)
ai_search_connection = client.connections.get("AzureAISearch")
# os.environ["AZURE_OPENAI_API_KEY"] = "0464ab68426e4f8bb1418052c16befc3" # workaround for connections.get() not returning api_key
embeddings_model_config = ModelConfiguration.from_connection(aoai_connection, 
                                                             model_name="text-embedding-ada-002",
                                                             deployment_name="text-embedding-ada-002")

## cohere
cohere_serverless_connection = client.connections.get("cohere-connection")
ai_search_connection = client.connections.get("AzureAISearch")
embeddings_model_config = ModelConfiguration.from_connection(cohere_serverless_connection)


## Input sources
input_source_local = LocalSource(input_data="product-info/")
input_source_git = GitSource(git_url="https://github.com/rust-lang/book.git", git_branch_name="main", git_connection_id="")
input_source_urls="azureml://subscriptions/f375b912-331c-4fc5-8e9f-2d7205e3e036/resourcegroups/rg-jingyizhuai/workspaces/jingyizhu-project-2/datastores/workspaceblobstore/paths/UI/2024-04-22_171136_UTC/"
input_source_ai_search = AISearchSource(ai_search_index_name="remote_index",
                                        ai_search_index_content_key="content",
                                        ai_search_index_embedding_key="contentVector",
                                        ai_search_index_title_key="title",
                                        ai_search_index_metadata_key="meta_json_string",
                                        ai_search_index_connection_id=ai_search_connection.id
                                        )

In [ ]:
client.indexes.build_index(
    name=index_name, # name of your index
    embeddings_model_config=embeddings_model_config,
    # data_source_citation_url="https://azuremlexamples.blob.core.windows.net/product-info", # citation links
    input_source=input_source_local,
    # input_source_credential=UserIdentityConfiguration(), # user specified identity used to access the data. Default behavior is to just assume identity of the ml_client
    index_config=AzureAISearchConfig(
        ai_search_index_name=index_name,  # the name of the index store in Ai search service
        ai_search_connection_id=ai_search_connection.id, # get the AI Search connection details
    ),
    # overwrite_if_exists=False, # default behavior is False
    tokens_per_chunk = 800, # Optional field - Maximum number of tokens per chunk
    token_overlap_across_chunks = 0, # Optional field - Number of tokens to overlap between chunks
)

In [ ]:
ml_index=client.indexes.get("remote-url", label="latest")

ml_index=client.indexes.get("jingyi-test-index", label="latest")

In [ ]:
# An mlindex can be converted to a LangChain retriever and used in a chain
retriever = ml_index.as_langchain_retriever()
# LangChain provides general query APIs
retriever.get_relevant_documents("which tent is the most waterproof?")

In [ ]:
# from langchain.chains import RetrievalQA

# completion_model = client.model_catalog.get_model('azure_open_ai://gpt-35-turbo')
# chain = RetrievalQA.from_chain_type(llm=completion_model, chain_type="stuff", retriever=retriever)

# user_input = 'which tent is the most waterproof?'
# result = chain.run(user_input)

In [ ]:
path = "azureml://subscriptions/f375b912-331c-4fc5-8e9f-2d7205e3e036/resourcegroups/rg-jingyizhuai/workspaces/jingyizhu-project-2/datastores/workspaceblobstore/paths/indexes/remote-local-02/3a76509a-600b-4c65-a593-1b0944fa68ff/"
from azureml.rag.mlindex import MLIndex as InternalMLIndex
retriever = InternalMLIndex(str(path)).as_langchain_retriever()
retriever.get_relevant_documents("which tent is the most waterproof?")